<div style="font-size:18px">Author: Himanshu Choudhary <br>
Git: https://github.com/choudharyhimanshu</div>

In [ ]:
%matplotlib inline
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from os import listdir

In [ ]:
GRID_W = 10
GRID_H = 10

GRID_IMG_W = 32
GRID_IMG_H = 32
DIR_GRID_IMGS = "grid-images/"

INP_IMG = "1.jpg"
INP_IMG_W = 50
INP_IMG_H = 50

OUT_IMG = "out.jpg"

In [ ]:
# RGB values average
def get_avg_col(img):
    avg = np.average(np.average(img,axis=0),axis=0)
    return np.uint8(avg)

def get_col_img(color):
    return np.array([[color]*10]*10, np.uint8)


In [ ]:
inp_img = cv2.resize(cv2.imread(INP_IMG), (INP_IMG_W,INP_IMG_H))
grid_imgs = []

for file in listdir(DIR_GRID_IMGS):
    grid_imgs.append((cv2.resize(cv2.imread(DIR_GRID_IMGS+file), (GRID_IMG_W,GRID_IMG_H)),file,0))

grid_imgs = pd.DataFrame(grid_imgs,columns=["img","filename","count"])

### Pixel wise average

In [ ]:
grid_imgs["avg_col"] = grid_imgs.img.map(get_avg_col)

In [ ]:
for i,row in grid_imgs.sample(5).iterrows():
    plt.imshow(row.img)
    plt.show()    
    plt.imshow(get_col_img(row.avg_col))
    plt.show()

In [ ]:
out_img = np.array([[[0,0,0] for _ in range(GRID_H*INP_IMG_H)] for _ in range(GRID_W*INP_IMG_W)])

In [ ]:
%%time
for i in range(INP_IMG_H):
    for j in range(INP_IMG_W):
        pix = inp_img[i][j]
        min_dist = float('inf')
        min_dist_img = None
        for index,row in grid_imgs.iterrows():
            dist = np.linalg.norm(pix - row.avg_col)
            if dist < min_dist:
                min_dist = dist
                min_dist_img = row.img
        grid = cv2.resize(min_dist_img,(GRID_W,GRID_H))
        for k in range(GRID_W):
            for l in range(GRID_H):
                out_img[i*GRID_W + k][j*GRID_H + l] = grid[k][l]

In [ ]:
cv2.imwrite(OUT_IMG,out_img)
plt.imshow(np.array(out_img,dtype=np.uint8))
plt.show()
plt.imshow(inp_img)
plt.show()